In [ ]:
from econml.iv.nnet import DeepIV
import torch

import keras
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

In [ ]:
alpha_values = [0, 1, 5]
runs = 10  # Number of runs for each alpha value

In [ ]:
treatment_model = keras.Sequential([keras.layers.Dense(256, activation='relu', input_shape=(2,)),
                                    keras.layers.Dropout(0.17),
                                    keras.layers.Dense(128, activation='relu'),
                                    keras.layers.Dropout(0.17),
                                    keras.layers.Dense(64, activation='relu'),
                                    keras.layers.Dropout(0.17),
                                    keras.layers.Dense(32, activation='relu'),
                                    keras.layers.Dropout(0.17)])

response_model = keras.Sequential([keras.layers.Dense(256, activation='relu', input_shape=(2,)),
                                   keras.layers.Dropout(0.17),
                                   keras.layers.Dense(128, activation='relu'),
                                   keras.layers.Dropout(0.17),
                                   keras.layers.Dense(64, activation='relu'),
                                   keras.layers.Dropout(0.17),
                                   keras.layers.Dense(32, activation='relu'),
                                   keras.layers.Dropout(0.17),
                                   keras.layers.Dense(1)])

In [ ]:
keras_fit_options = { "epochs": 30,
                      "validation_split": 0.2,
                      "callbacks": [keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)]}

deepIvEst = DeepIV(n_components = 10, # number of gaussians in our mixture density network
                   m = lambda z, x : treatment_model(keras.layers.concatenate([z,x])), # treatment model
                   h = lambda t, x : response_model(keras.layers.concatenate([t,x])),  # response model
                   n_samples = 1, # number of samples to use to estimate the response
                   use_upper_bound_loss = False, # whether to use an approximation to the true loss
                   n_gradient_samples = 1, # number of samples to use in second estimate of the response (to make loss estimate unbiased)
                   optimizer='adam', # Keras optimizer to use for training - see https://keras.io/optimizers/ 
                   first_stage_options = keras_fit_options, # options for training treatment model
                   second_stage_options = keras_fit_options) # options for training response model

In [ ]:
for alpha in alpha_values:
    train_data_path = f'data/sec5.4/train_Zcont_g_fct_f_softplusalpha{alpha}.csv'
    test_data_path = f'data/sec5.4/test_Zcont_g_fct_f_softplusalpha{alpha}.csv'
    
    train_data = np.genfromtxt(train_data_path, delimiter=',', skip_header=1)
    test_data = np.genfromtxt(test_data_path, delimiter=',', skip_header=1)
    
    data_length = train_data.shape[0]

    # Separate the columns into individual tensors
    z = train_data[:, 0]
    t = train_data[:, 1]
    y = train_data[:, 2]
    x = np.zeros(data_length)
    
    X_test = torch.tensor(test_data[:,0].astype(np.float32)).squeeze()
    #X_test_grid = torch.tensor(test_data[:,2].astype(np.float32)).squeeze()

    df_mse = pd.DataFrame()
    #df_plot = pd.DataFrame()
    
    for run in range(runs):
        # Fit the DeepIV model
        deepIvEst.fit(Y=y, T=t, X=x, Z=z)
        
        # Generate predictions for each x_temp
        for i, x_temp in enumerate([x[0]]):
            y_hat_deepiv = deepIvEst.predict(X_test, np.full_like(X_test, x_temp))
            #y_hat_deepiv_grid = deepIvEst.predict(X_test_grid, np.full_like(X_test_grid, x_temp))

        # Add the predictions as a new column to the DataFrame
        df_mse[f'Run_{run+1}'] = y_hat_deepiv
        #df_plot[f'Run_{run+1}'] = y_hat_deepiv_grid
        
    # Save the results for the current alpha
    df_mse.to_csv(f'results/sec5.4/deepiv_result_mse_alpha{alpha}.csv', index=False)
    #df_plot.to_csv(f'results/deepiv_result_plot_alpha{alpha}.csv', index=False)
    
    # Optionally, plot the results
    # plt.scatter(X_test_grid, df_plot.iloc[:, -1], s=1, color='red', label=f'DeepIV Alpha {alpha}')
    # plt.scatter(X_test_grid, test_data[:, 3], s=1)
    # plt.legend()
    # plt.show()
